In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

In [2]:
# Step 1 - load the data and explore
data3 = pd.read_csv("../raw-data/yeast_data.csv")

# drop the first column, which only serves as a row index
data3 = data3.iloc[:, 1:]

# drop amino acid identity one-hot encoding columns (first 180 after dropping the index column)
data3 = data3.iloc[:, 180:]

# set the random seed
random_seed = 1

# separate the majority and minority classes
majority = data3[data3['target_value'] == 0]
minority = data3[data3['target_value'] == 1]

# downsample the majority class
majority_downsampled = resample(
    majority,
    replace=False,            
    n_samples=7500,           
    random_state=random_seed
)

# downsample the majority class
minority_downsampled = resample(
    minority,
    replace=False,            
    n_samples=7500,           
    random_state=random_seed
)

# combine the minority class with the downsampled majority class
balanced = pd.concat([majority_downsampled, minority_downsampled])

# shuffle the data
balanced = balanced.sample(frac=1, random_state=random_seed).reset_index(drop=True)

print(balanced['target_value'].value_counts())

balanced.describe()

target_value
1    7500
0    7500
Name: count, dtype: int64


,A_-4_pssm,R_-4_pssm,N_-4_pssm,D_-4_pssm,C_-4_pssm,Q_-4_pssm,E_-4_pssm,G_-4_pssm,H_-4_pssm,I_-4_pssm,...,KARS160118_aaindex,KARS160119_aaindex,KARS160120_aaindex,KARS160121_aaindex,KARS160122_aaindex,str__AlphaHelix,str__Coil,str__Strand,str__Turn,target_value
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,...,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,-0.009423,-0.000123,0.014152,-0.008416,-0.002473,-0.010411,-0.027207,0.034204,-0.006342,-0.029456,...,-0.057133,-0.067266,-0.047410,-0.056217,0.063451,-0.036926,0.016695,0.029942,0.016429,0.500000
std,0.993031,1.007926,1.009345,0.985395,1.006521,1.006512,0.970355,1.042930,0.995706,1.002494,...,1.023094,1.010846,1.019811,1.019033,1.006637,1.024160,1.261314,1.026263,1.032122,0.500017
min,-3.702108,-2.859091,-3.347503,-2.806862,-2.932563,-2.958117,-2.867022,-2.511551,-2.623656,-2.566789,...,-4.417969,-4.630692,-5.854570,-4.591849,-2.790285,-1.888825,-0.028222,-0.444558,-0.236844,0.000000
25%,-0.745324,-0.760338,-0.816681,-0.636419,-0.437336,-0.613271,-0.831815,-0.519308,-0.753772,-0.661201,...,-0.709607,-0.682777,-0.655088,-0.682178,-0.658834,0.529430,-0.028222,-0.444558,-0.236844,0.000000
50%,-0.322926,-0.410545,-0.093589,-0.326355,-0.021465,-0.222463,-0.153413,-0.187267,-0.005819,-0.343603,...,-0.015409,0.028585,0.124381,0.036310,-0.024865,0.529430,-0.028222,-0.444558,-0.236844,0.500000
75%,0.521869,0.289039,0.629503,0.603835,0.394407,0.559152,0.524990,0.476814,0.368158,0.609192,...,0.606765,0.685694,0.713759,0.680645,0.704375,0.529430,-0.028222,-0.444558,-0.236844,1.000000
max,3.478653,3.786962,3.883417,3.394404,6.216604,4.076420,3.238599,3.465178,4.855880,3.149976,...,4.177504,2.337356,1.450158,3.436936,5.218115,0.529430,35.432936,2.249425,4.222185,1.000000


In [3]:
# Separate the feature columns and the target column
features = balanced.drop(columns=['target_value'])
outcome  = balanced['target_value']

# Initialize the scaler
scaler = StandardScaler()

# Scale the feature columns
scaled_features = scaler.fit_transform(features)

# Create a new DataFrame with scaled features and the target column
scaled_ = pd.DataFrame(scaled_features, columns=features.columns)
scaled_['target_value'] = outcome.reset_index(drop=True)

# Print to verify
print(scaled_.head())

scaled_.describe()

   A_-4_pssm  R_-4_pssm  N_-4_pssm  D_-4_pssm  C_-4_pssm  Q_-4_pssm  \
0  -1.166467   3.063321  -0.464957  -0.637332  -1.258442   0.177605   
1  -0.315714  -0.407209   0.251464   2.509363   0.807513   0.177605   
2  -1.166467  -1.448368  -1.539589  -1.581340  -0.432060  -1.375564   
3   0.960415   0.633950   1.326096   0.306677  -0.018869   0.954189   
4  -0.741091  -0.407209  -0.464957  -0.637332  -0.432060  -0.598979   

   E_-4_pssm  G_-4_pssm  H_-4_pssm  I_-4_pssm  ...  KARS160118_aaindex  \
0  -0.130065  -0.530745  -0.375076  -0.947012  ...           -0.183969   
1   1.967388   0.106022  -0.375076   0.003443  ...           -0.346913   
2  -1.528368  -1.485896  -1.501882   1.904353  ...            0.328332   
3   0.569086   2.016322   0.000526  -0.630194  ...            0.681985   
4  -0.479641  -0.530745   1.127332  -0.313376  ...            0.371299   

   KARS160119_aaindex  KARS160120_aaindex  KARS160121_aaindex  \
0           -0.513191            0.575881           -0.238002  

,A_-4_pssm,R_-4_pssm,N_-4_pssm,D_-4_pssm,C_-4_pssm,Q_-4_pssm,E_-4_pssm,G_-4_pssm,H_-4_pssm,I_-4_pssm,...,KARS160118_aaindex,KARS160119_aaindex,KARS160120_aaindex,KARS160121_aaindex,KARS160122_aaindex,str__AlphaHelix,str__Coil,str__Strand,str__Turn,target_value
count,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,...,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,15000.000000
mean,6.631732e-18,-9.473903e-19,-1.065814e-17,-4.357995e-17,-3.647453e-17,-1.231607e-17,-4.310626e-17,-1.894781e-18,-1.894781e-18,-6.631732e-18,...,-3.789561e-18,-2.569796e-17,7.105427e-18,1.539509e-18,-2.273737e-17,-1.117921e-16,-1.989520e-17,7.034373e-17,6.347515e-17,0.500000
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,...,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,0.500017
min,-3.718725e+00,-2.836580e+00,-3.330643e+00,-2.840018e+00,-2.911205e+00,-2.928732e+00,-2.926670e+00,-2.441046e+00,-2.628689e+00,-2.531105e+00,...,-4.262544e+00,-4.514611e+00,-5.694541e+00,-4.451064e+00,-2.835015e+00,-1.808274e+00,-3.561282e-02,-4.623728e-01,-2.453987e-01,0.000000
25%,-7.410907e-01,-7.542619e-01,-8.231681e-01,-6.373316e-01,-4.320599e-01,-5.989794e-01,-8.292166e-01,-5.307452e-01,-7.506783e-01,-6.301941e-01,...,-6.377679e-01,-6.089267e-01,-5.958938e-01,-6.142900e-01,-7.175470e-01,5.530135e-01,-3.561282e-02,-4.623728e-01,-2.453987e-01,0.000000
50%,-3.157144e-01,-4.072089e-01,-1.067468e-01,-3.226621e-01,-1.886905e-02,-2.106873e-01,-1.300654e-01,-2.123618e-01,5.258429e-04,-3.133757e-01,...,4.078276e-02,9.482615e-02,1.684588e-01,9.080126e-02,-8.773702e-02,5.530135e-01,-3.561282e-02,-4.623728e-01,-2.453987e-01,0.500000
75%,5.350384e-01,2.868972e-01,6.096745e-01,6.213463e-01,3.943218e-01,5.658969e-01,5.690857e-01,4.244051e-01,3.761279e-01,6.370796e-01,...,6.489334e-01,7.449061e-01,7.464071e-01,7.231234e-01,6.367197e-01,5.530135e-01,-3.561282e-02,-4.623728e-01,-2.453987e-01,1.000000
max,3.512673e+00,3.757428e+00,3.833570e+00,3.453372e+00,6.178994e+00,4.060526e+00,3.365690e+00,3.289856e+00,4.883353e+00,3.171627e+00,...,4.139189e+00,2.378900e+00,1.468525e+00,3.428023e+00,5.120849e+00,5.530135e-01,2.807977e+01,2.162757e+00,4.075001e+00,1.000000


In [4]:
scaled_.to_csv('../raw-data/yeast-processed_v2.csv', index=False)